## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
###Drop Nan Values
df=df.dropna()


In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.9.1'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [20]:
messages.reset_index(inplace=True)

In [21]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kartikay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2494, 1087, 3363, 2187, 2143, 3555, 142, 582, 3047, 2932],
 [783, 4825, 4083, 552, 2031, 3349, 1960],
 [1041, 3627, 4588, 2183],
 [651, 3380, 801, 3466, 1647, 4579],
 [4296, 2031, 3521, 2510, 2728, 2220, 2031, 2695, 4650, 844],
 [3973,
  4980,
  2180,
  1413,
  4074,
  930,
  3956,
  3992,
  1452,
  393,
  2557,
  1980,
  578,
  555,
  1960],
 [2881, 4549, 1128, 3107, 2404, 4874, 1481, 3843, 2159, 4053, 1620],
 [4061, 2252, 2192, 2892, 4596, 4980, 930, 3113, 2159, 4053, 1620],
 [3415, 2228, 4409, 4500, 2605, 1745, 3424, 2492, 930, 2501],
 [4946, 1922, 535, 2996, 2386, 2715, 1940, 4101],
 [4303, 3993, 3098, 3656, 234, 1665, 633, 3774, 1013, 4772, 3958],
 [3466, 1109, 2143, 1745, 930, 4596],
 [2673, 1640, 1857, 1508, 2728, 1308, 4672, 1002, 1654],
 [104, 2852, 1208, 2054, 3734, 1022, 400, 2159, 4053, 1620],
 [4597, 1526, 1492, 1379, 1133, 2159, 4053, 1620],
 [3840, 956, 4083, 3427, 925, 3863, 4560, 1201, 4754, 1476],
 [4075, 3569, 4825],
 [1700, 3403, 2770, 3538, 930, 4762, 3991, 1960]

In [28]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
onehot_repr[1]

[783, 4825, 4083, 552, 2031, 3349, 1960]

### Embedding Representation

In [30]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  582 3047 2932]
 [   0    0    0 ... 2031 3349 1960]
 [   0    0    0 ... 3627 4588 2183]
 ...
 [   0    0    0 ... 2159 4053 1620]
 [   0    0    0 ... 1248 4412 2850]
 [   0    0    0 ... 2048 3724 3935]]


In [31]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  783, 4825, 4083,  552, 2031, 3349, 1960])

In [32]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2494,
       1087, 3363, 2187, 2143, 3555,  142,  582, 3047, 2932])

In [33]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
len(embedded_docs),y.shape

(18285, (18285,))

In [35]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [47]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 42ms/step - loss: 0.3493 - accuracy: 0.8309 - val_loss: 0.1987 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1507 - accuracy: 0.9384 - val_loss: 0.1996 - val_accuracy: 0.9138
Epoch 3/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1101 - accuracy: 0.9586 - val_loss: 0.2062 - val_accuracy: 0.9135
Epoch 4/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0833 - accuracy: 0.9698 - val_loss: 0.2438 - val_accuracy: 0.9162
Epoch 5/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0645 - accuracy: 0.9768 - val_loss: 0.2577 - val_accuracy: 0.9080
Epoch 6/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0476 - accuracy: 0.9841 - val_loss: 0.3202 - val_accuracy: 0.9152
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0336 - accuracy: 0.9896 - val_loss: 0.3808 - val_accuracy: 0.916

### Adding Dropout 

In [39]:
## dropout reduces the overfitting

from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [48]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 7ms/step


In [49]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 234, 2382]], dtype=int64)

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9111847555923778

In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

